Bailie Geddes, 17beg, 20099792

# **Implementing ResNet for Image Classification:**

This notebook has portions taken from published source on Kaggle and has been extended by modifying:
- add image data generator for all pre-processing functions
    - rotation, width shift, height shift, horizontal flip, brightness, etc.
- changing the ResNet50 model by adding additional dense layers
    - add dense layers for additional simple layers of neurons
    - add new dropout layer
- changing the classification by altering the class weights 
    - redistributes the balance of images in each class, adding classweights by calculating proportions and inversing them
- adding batch normalization
    - standardizes the inputs to a layer for each mini-batch to reduce number of epochs needed
  

The dataset that will be worked with is the State Farm Distracted Driver Detection dataset found on Kaggle.com (https://www.kaggle.com/c/state-farm-distracted-driver-detection/data).

There are 10 different classes, and 79.7 thousand images. The classes are safe driving, texting with right hand, talking on the phone with right hand, texting with left hand, talking on the phone with left hand, operating the radio, drinking, reaching behind, hair and makeup and talking to passenger. 

Using this Dataset, I’m going to present results of Residual neural networks (ResNet) used for Image classification to test the accuracy they present for these images, first creating it piece by piece and then importing and adapting a pre trained ResNet.

# 1. Import libraries

In [ ]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageEnhance
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
tf.__version__

# 2. Get data

In [ ]:
# sample_path = "/kaggle/input/state-farm-distracted-driver-detection/sample_submission.csv"
imgs_list_path = "/kaggle/input/state-farm-distracted-driver-detection/driver_imgs_list.csv"
train_path = "/kaggle/input/state-farm-distracted-driver-detection/imgs/train"

# 3.Check data distribution

In [ ]:
# read csv file to get class names
driver_imgs_list = pd.read_csv(imgs_list_path)
driver_imgs_list.head()

In [ ]:
os.listdir(train_path)

In [ ]:
# sort through to organize dataset by class rather then random
def pair_sort(className,values):
    for j in range(0,len(className)-1):
        for i in range(0,len(className)-1):
            if values[i] > values[i+1]:
                temp =  values[i+1]
                values[i+1] = values[i]
                values[i] = temp

                N_temp =  className[i+1]
                className[i+1] = className[i]
                className[i] = N_temp
    
    return className,values

In [ ]:
# create graph to display class weight and number of images in each class
from matplotlib.pyplot import figure
figure(num=None, figsize=(15, 5), dpi=80, facecolor='w', edgecolor='k')

class_names = np.unique(driver_imgs_list['classname'])
class_image_list = [len(driver_imgs_list[driver_imgs_list['classname'] == current_class]) for current_class in class_names]

class_names,class_image_list=  pair_sort(class_names,class_image_list)

#plt.figure()
plt.suptitle('Number of images per Class')
plt.bar(class_names,class_image_list,color=(0.2, 0.4, 0.6, 0.6))
plt.show()

In [ ]:
# create graph to display number of images per subject
from matplotlib.pyplot import figure
sub_names = np.unique(driver_imgs_list['subject'])
sub_image_list = [len(driver_imgs_list[driver_imgs_list['subject'] == current_sub]) for current_sub in sub_names]
sub_names,sub_image_list=  pair_sort(sub_names,sub_image_list)

figure(num=None, figsize=(15, 10), dpi=80, facecolor='w', edgecolor='k')

y_pos = np.arange(len(sub_names))
# Create horizontal bars
plt.barh(y_pos, sub_image_list,color=(0.2, 0.4, 0.6, 0.6))
 
# Create names on the y-axis
plt.yticks(y_pos,sub_names )
plt.suptitle('Number of images per subject')

# Show graphic
plt.show()

In [ ]:
# hyperparameters
img_width,img_height = (256,256)
model_input_shape = (img_width,img_height,3)
batch_size = 16
input_image = (img_width,img_height)

# get image path from dataset
def load_image(path):
    read_path = train_path+"/"+path
    image = Image.open(read_path)
    image = image.resize(input_image)
    
    return np.asarray(image)

In [ ]:
# load image pixels from input data
def show_images(image_ids,class_names):
    pixels = [load_image(path) for path in image_ids]
    num_of_images = len(image_ids)
    fig, axes = plt.subplots(
        1, 
        num_of_images, 
        figsize=(5 * num_of_images, 5 * num_of_images),
        
    )
   
    # convert enumeration value to integer
    for i, image_pixels in enumerate(pixels):
        axes[i].imshow(image_pixels)
        axes[i].axis("off")
        axes[i].set_title(class_names[i])

# 4.Plot class images

In [ ]:
# checking input data
sub_names_imgs = [ current_class+"/"+driver_imgs_list[driver_imgs_list['classname'] == current_class]['img'].values[0] for current_class in class_names]

show_images(sub_names_imgs[:5],class_names[:5])
show_images(sub_names_imgs[5:],class_names[5:])

# 5. Split and load Train/Validation 

In [ ]:
# get test and train data
train_path = "/kaggle/input/state-farm-distracted-driver-detection/imgs/train"
test_path = "/kaggle/input/state-farm-distracted-driver-detection/imgs/test"

In [ ]:
# x_train = []
# y_train = []

# x_val = []
# y_val = []

# # The data is split into a test set and a training set
# # a driver can only appear in one of the training set or the testing set
# split_rate = 0.8

# # go through dataset and collecting train and test data suing split factor
# # training neural network
# for current_class in class_names:
#     select_df = driver_imgs_list[driver_imgs_list['classname'] == current_class ]
#     image_list = select_df['img'].values
#     train_amount = int(len(image_list)*split_rate)
#     train_list = image_list[:train_amount]
#     val_list = image_list[train_amount:]
    
#     for filename in train_list:
#         x_train.append(load_image(current_class+"/"+filename))
#         y_train.append(current_class.replace('c',''))

#     for filename in val_list:
#         x_val.append(load_image(current_class+"/"+filename))
#         y_val.append(current_class.replace('c',''))


In [ ]:
# search dataset to organize
driver_imgs_list["img_path"] = driver_imgs_list["classname"]+"/"+driver_imgs_list["img"]
driver_imgs_list

In [ ]:
# display data
labels = driver_imgs_list["classname"]
data_x = driver_imgs_list["img_path"]
train_df, val_df = train_test_split(driver_imgs_list, test_size = 0.2)

In [ ]:
# display data
len(train_df), len(val_df)
val_df.head()

# 6. Image Pre-processing

In [ ]:
# image pre-processing using data generator
datagen = ImageDataGenerator(
        rotation_range=10, # rotation
        width_shift_range=0.2, # horizontal shift
        height_shift_range=0.2, # vertical shift
        zoom_range=0.2, # zoom
        horizontal_flip=True, # horizontal flip
        brightness_range=[0.2,1.2]) # brightness

# process train data
train_generator_df = datagen.flow_from_dataframe(dataframe=train_df, 
                                              directory=train_path+"/",
                                              x_col="img_path", 
                                              y_col="classname", 
                                              class_mode="categorical", 
                                              target_size=(256, 256), 
                                              batch_size=32,
                                              rescale=1.0/255)
# process val data
val_generator_df = datagen.flow_from_dataframe(dataframe=val_df, 
                                              directory=train_path+"/",
                                              x_col="img_path", 
                                              y_col="classname", 
                                              class_mode="categorical", 
                                              target_size=(256, 256), 
                                              batch_size=32,
                                              rescale=1.0/255)

## 7. Encode Labels

In [ ]:
# # convert vector to matrix
# x_train = np.asarray(x_train)
# y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
# x_val = np.asarray(x_val)
# y_val =tf.keras.utils.to_categorical(y_val, num_classes=10)

# print("Train x Shape: ",x_train.shape)
# print("Test x Shape: ",x_val.shape)


In [ ]:
# print("Train y Shape: ",y_train.shape)
# print("Test y Shape: ",y_val.shape)

## 8. Create Model


In [ ]:
# create our Resnet model
# include_top is false to allow us to modify classification
base_model  = tf.keras.applications.resnet.ResNet50(include_top = False,
                                                  weights = 'imagenet',
                                                  input_shape = model_input_shape)
# base_model.summary()

In [ ]:
# x = base_model.output
# x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.Dropout(0.5)(x)

# output =tf.keras.layers.Dense(units = len(class_names),activation = tf.nn.softmax)(x)
# model = tf.keras.models.Model(inputs=base_model.inputs, outputs=output)

# model.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
#               loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False),
#               metrics=['accuracy'])

# model.summary()

In [ ]:
# calculating class weights which are inversely proportional to number of training examples
classes, counts = np.unique(train_df["classname"], return_counts = True)
total = sum(counts)
ratios = 1/(counts/total)

class_weights = dict()

for i in range(10):
    class_weights[i] = ratios[i]

In [ ]:
class_weights

In [ ]:
#modified model of ResNet
x = base_model.output
# flattens input layer to pass  data into every single neuron of the model 
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(units = 1068, activation = tf.nn.relu)(x)
# help reduce overfitting
x = tf.keras.layers.Dropout(0.25)(x)
# additional dense layers added
x = tf.keras.layers.Dense(units = 768, activation = tf.nn.relu)(x)
# normalize flow of data to increase learning rate
x = tf.keras.layers.BatchNormalization()(x)
# feeds output to neurons from previous layer
x = tf.keras.layers.Dense(units = 512, activation = tf.nn.relu)(x)
x = tf.keras.layers.Dense(units = 256, activation = tf.nn.relu)(x)
x = tf.keras.layers.Dropout(0.25)(x)

output =tf.keras.layers.Dense(units = len(class_names),activation = tf.nn.softmax)(x)
model = tf.keras.models.Model(inputs=base_model.inputs, outputs=output)

# Adam = optimization algorithm for stochastic gradient descent - help handle noise
model.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False),
              metrics=['accuracy'])

model.summary()

In [ ]:
num_epochs = 40
def lr_schedule(epoch,lr):
    # Learning Rate Schedule

    lr = lr
    total_epochs = num_epochs

    # reduce the learning rate as as epochs increase
    check_1 = int(total_epochs * 0.9)
    check_2 = int(total_epochs * 0.8)
    check_3 = int(total_epochs * 0.6)
    check_4 = int(total_epochs * 0.4)

    if epoch > check_1:
        lr *= 1e-4
    elif epoch > check_2:
        lr *= 1e-3
    elif epoch > check_3:
        lr *= 1e-2
    elif epoch > check_4:
        lr *= 1e-1

    print("[+] Current Lr rate : {} ".format(lr))
    return lr
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

In [ ]:
# # setting hyperparameters for using with model
# history = model.fit(
#       x = x_train,y=y_train,
#       validation_data=(x_val,y_val),
#       steps_per_epoch=141, # will reduce the training data to 128 images per class
#       batch_size = 40,
#       epochs=75,
#     class_weight=class_weights,
#     callbacks = [lr_callback],
#       verbose=1)

# 9. Train and Test

In [ ]:
# setting hyperparameters for using with model
history = model.fit_generator(
      train_generator_df,
      validation_data=val_generator_df,
      # usually 100 - for testing we will use 50
      steps_per_epoch=100, # will reduce the training data to 128 images per class
#       batch_size = 40,
      # usually epoch = 40 - for testing we will use 4
      epochs=40,
    class_weight=class_weights,
    callbacks = [lr_callback],
      verbose=1)

## 10. Model Evaluation

In [ ]:
# # create graphs to display training and validation loss and accuracy of the model
# fig, ax = plt.subplots(1, 2, figsize=(15, 5))

# ax[0].set_title('Training vs Validation Accuracy')
# ax[0].plot(history.history['accuracy'])
# ax[0].plot(history.history['val_accuracy'])
# plt.ylabel('Accuracy')
# plt.xlabel('Epochs')
# plt.show()

# ax[1].set_title('Training vs Validation Loss')
# ax[1].plot(history.history['loss'])
# ax[1].plot(history.history['val_loss'])
# plt.ylabel('Loss')
# plt.xlabel('Epochs')

In [ ]:
# create graphs to display training and validation loss of the model
loss_train = history.history['loss']
loss_val = history.history['val_loss']
# usually up to 41 but for testing changing it to 5
epochs = range(1,41)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc = "upper right")
plt.show()

In [ ]:
# create graphs to display training and validation accuracy of the model
loss_train = history.history['accuracy']
loss_val = history.history['val_accuracy']
# usually up to 41 but for testing changing it to 5
epochs = range(1,41)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc = "upper right")
plt.show()

# 11. Prediction

In [ ]:
#  display the evaluation results of prediction using part of dataset
y_pred = model.evaluate_generator(val_generator_df) #predict on x_val
# y_pred = np.argmax(y_pred, axis = 1) # convert output to class numbers
# print("Predictions:", y_pred)
# print("True labels:", np.argmax(y_val, axis = 1)) # print the prediction and true labels for first 10 test images

In [ ]:
# our prediction accuracy using sample set
print("Accuracy on test set")
print(y_pred[1])

In [ ]:
# create graphs to display prediction of sample data
val = history.history['accuracy']
# usually up to 41 but for testing changing it to 5
epochs = range(1,41)
plt.plot(epochs, val, 'g', label='Training accuracy')
plt.title('Prediction Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc = "lower right")
plt.show()